In [1]:
import numpy as np
from scipy import sparse
from voxcell import VoxelData
from voxcell.nexus.voxelbrain import Atlas

In [2]:
atlas_dir = '../data/atlas/25um/'
hierarchy_path = f'{atlas_dir}/hierarchy.json'

In [3]:
atlas = Atlas.open(atlas_dir)
region_map = atlas.load_region_map()

In [32]:
 = atlas.get_region_mask('CA1')

In [5]:
root_children = list(region_map.find('root','name',with_descendants=True))
mask_hierarchy_indices = [region_map.get(i,'st_level') == 1 for i in root_children]

In [22]:
st_lvl_1_ids = np.array(root_children)[list(np.where(mask_hierarchy_indices)[0])]

In [25]:
st_lvl_1_ids

array([        8,        73,      1009,      1024, 304325711])

In [23]:
[region_map.get(i,'name') for i in st_lvl_1_ids]

['Basic cell groups and regions',
 'ventricular systems',
 'fiber tracts',
 'grooves',
 'retina']

In [28]:
vd = VoxelData.load_nrrd(f'{atlas_dir}/brain_regions.nrrd')
vd_new_raw = np.zeros_like(vd.raw).astype(int)
# set all values to -1 but keep the same shape
#vd_new_raw[:] = -1

for i in st_lvl_1_ids:
    region_name = region_map.get(i,'name') 
    print(region_name, i)
    mask = atlas.get_region_mask(i,attr='id').raw.astype(int) * i
    assert np.all(mask[mask != 0] == i)
    vd_new_raw += mask


Basic cell groups and regions 8
ventricular systems 73
fiber tracts 1009
grooves 1024
retina 304325711


In [29]:
np.unique(vd_new_raw)

array([   0,    8,   73, 1009])

In [40]:
# save new mask
brain_regions = atlas.load_data("brain_regions")
st_lvl_1 = brain_regions.with_data(np.asarray(vd_new_raw, dtype=np.uint64))
np.testing.assert_array_equal(np.unique(st_lvl_1.raw),np.unique(vd_new_raw))
st_lvl_1.save_nrrd(f'{atlas_dir}/brain_regions_st_lvl_1.nrrd')